This notebook compares the overfitting of fairlearn Vs OxonFair using random forests and decision trees on the adult dataset.

We use sex as the protected attribute.

Even on this low-dimensional data, the default parameters of scikit-learn cause both decision trees and random forests to overfit. 

The models obtain 0 error on the training set. As a consequence of this, definitions such as equal opportunity are trivially satisfied, and fairness methods such as fairlearn which enforce fairness on the training set do not work.

This overfitting, and the consequential failure of fairness methods to work can be avoided by specifying a low maximal tree depth. The examples in fairlearn documentation typically use a tree depth of 4 on adult. 

OxonFair allows for the enforcing of fairness on validation data, and this means that it can enforce fairness even when the training error is zero. 

In [1]:
from oxonfair import FairPredictor, performance, dataset_loader
from oxonfair import group_metrics as gm
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train,val,test = dataset_loader.adult()
basetree = DecisionTreeClassifier().fit(X=train['data'], y=train['target'])
baseforest = RandomForestClassifier().fit(X=train['data'], y=train['target'])

We now specify a fair predictor over the validation set.

In [3]:
# The outputs of a decision tree are all 0 or 1, so we add Gaussian noise to allow thresholding to work
ftree=FairPredictor(basetree,val,add_noise=0.001)
fforest=FairPredictor(baseforest,val)

We call fit to enforce equal opportunity.

In [4]:
ftree.fit(gm.accuracy,gm.equal_opportunity,0.02)
fforest.fit(gm.accuracy,gm.equal_opportunity,0.02)

We now focus on trees only.
And evaluate fairness on validation data.

In [5]:
ftree.evaluate_fairness()

,original,updated
Statistical Parity,0.200729,0.161794
Predictive Parity,0.095843,0.095927
Equal Opportunity,0.091404,0.021044
Average Group Difference in False Negative Rate,0.091404,0.021044
Equalized Odds,0.101773,0.055598
Conditional Use Accuracy,0.099629,0.111880
Average Group Difference in Accuracy,0.125470,0.134539
Treatment Equality,0.012277,0.281679


And on the test set.

In [6]:
ftree.evaluate_fairness(test)

,original,updated
Statistical Parity,0.192595,0.148528
Predictive Parity,0.088080,0.088552
Equal Opportunity,0.061808,0.014514
Average Group Difference in False Negative Rate,0.061808,0.014514
Equalized Odds,0.085363,0.049967
Conditional Use Accuracy,0.099167,0.113456
Average Group Difference in Accuracy,0.129501,0.141496
Treatment Equality,0.114719,0.368360


We now check validation performance.

In [7]:
ftree.evaluate()

,original,updated
Accuracy,0.811384,0.803767
Balanced Accuracy,0.748654,0.715969
F1 score,0.614561,0.571837
MCC,0.489989,0.445613
Precision,0.601376,0.598355
Recall,0.628337,0.547570
ROC AUC,0.748654,0.732474


And on the test set.

In [8]:
ftree.evaluate(test)

,original,updated
Accuracy,0.810663,0.802883
Balanced Accuracy,0.744658,0.713858
F1 score,0.609723,0.568715
MCC,0.484844,0.442154
Precision,0.601599,0.596841
Recall,0.618070,0.543121
ROC AUC,0.744658,0.727113


We now run fairlearn on the same data.

In [9]:
from fairlearn.reductions import TruePositiveRateParity, ExponentiatedGradient
mitagator = ExponentiatedGradient(DecisionTreeClassifier(),TruePositiveRateParity())
mitagator.fit(X=train['data'],y=train['target'],sensitive_features=train['data']['sex'])

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:214: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.pos_basis[i]["+", e, g] = 1
/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fai

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.TruePositiveRateParity object at 0x1649162c0>,
                      estimator=DecisionTreeClassifier(), nu=0.0)

To evaluate fairlearn, we write a helper function to evaluate performance and fairness on train or test, and concatenate the outputs together.  

In [10]:
def eval(train, classifier=mitagator):
    return pd.concat((performance.evaluate(train['target'], classifier.predict(train['data'])),
                      performance.evaluate_fairness(train['target'], classifier.predict(train['data']), 
                                                    train['data'][train['groups']])),axis=0)

out = pd.concat((eval(train), eval(test)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,1.000000,0.808697
Balanced Accuracy,1.000000,0.741724
F1 score,1.000000,0.605405
MCC,1.000000,0.479245
Precision,1.000000,0.597732
Recall,1.000000,0.613279
ROC AUC,1.000000,0.741724
Statistical Parity,0.194578,0.197648
Predictive Parity,0.000000,0.070381
Equal Opportunity,0.000000,0.064160


Evaluating the initially trained baseline classifier we find that, as expected, fairlearn did not substantially alter the performance or unfairness of the classifier (beyond altering the random seed of the tree).

In [11]:
out = pd.concat((eval(train, basetree), eval(test, basetree)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,1.000000,0.810663
Balanced Accuracy,1.000000,0.744658
F1 score,1.000000,0.609723
MCC,1.000000,0.484844
Precision,1.000000,0.601599
Recall,1.000000,0.618070
ROC AUC,1.000000,0.744658
Statistical Parity,0.194578,0.192595
Predictive Parity,0.000000,0.088080
Equal Opportunity,0.000000,0.061808


We now do the same with the random forest classifier.

In [12]:
fforest.evaluate_fairness()

,original,updated
Statistical Parity,0.184390,0.166026
Predictive Parity,0.019392,0.061594
Equal Opportunity,0.106631,0.012080
Average Group Difference in False Negative Rate,0.106631,0.012080
Equalized Odds,0.092660,0.039273
Conditional Use Accuracy,0.057711,0.081734
Average Group Difference in Accuracy,0.111285,0.105621
Treatment Equality,0.256477,0.186913


In [13]:
fforest.evaluate_fairness(test)

,original,updated
Statistical Parity,0.184497,0.167118
Predictive Parity,0.025649,0.054314
Equal Opportunity,0.109656,0.016784
Average Group Difference in False Negative Rate,0.109656,0.016784
Equalized Odds,0.094767,0.043454
Conditional Use Accuracy,0.061651,0.079540
Average Group Difference in Accuracy,0.113465,0.110246
Treatment Equality,0.267356,0.137744


In [14]:
fforest.evaluate()

,original,updated
Accuracy,0.854464,0.854791
Balanced Accuracy,0.771340,0.783518
F1 score,0.668037,0.680713
MCC,0.579837,0.588426
Precision,0.735500,0.718358
Recall,0.611910,0.646817
ROC AUC,0.903369,0.889127


In [15]:
fforest.evaluate(test)

,original,updated
Accuracy,0.852756,0.850708
Balanced Accuracy,0.767633,0.776492
F1 score,0.662664,0.670284
MCC,0.573967,0.575758
Precision,0.733389,0.710779
Recall,0.604381,0.634155
ROC AUC,0.900784,0.884691


In [16]:
mitagator = ExponentiatedGradient(RandomForestClassifier(), TruePositiveRateParity())
mitagator.fit(X=train['data'], y=train['target'], sensitive_features=train['data']['sex'])

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:214: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.pos_basis[i]["+", e, g] = 1
/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fai

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.TruePositiveRateParity object at 0x164916e30>,
                      estimator=RandomForestClassifier(), nu=0.0)

In [17]:
out = pd.concat((eval(train,mitagator), eval(test,mitagator)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,1.000000,0.853820
Balanced Accuracy,1.000000,0.769271
F1 score,1.000000,0.665292
MCC,1.000000,0.577209
Precision,1.000000,0.735794
Recall,1.000000,0.607118
ROC AUC,1.000000,0.769271
Statistical Parity,0.194578,0.186343
Predictive Parity,0.000000,0.022945
Equal Opportunity,0.000000,0.120878
